# RTS dispatch Example

This notebook is intended to demonstrate some of the basic capability enabled by SIIP::Power. Specifically, this notebook gives some examples of how SIIP::Power can be used to conduct *Unit Commitment* on the RTS system



## Packages

 - *[PowerSystems.jl](https://github.com/NREL/PowerSystems.jl)* We take advantage of Julia's dynamic types and functional dispatch in our implementation of PowerSystems to define data schemas for Power Systems Analysis problems.

 - *[PowerSimulations.jl](https://github.com/NREL/PowerSimulations.jl)* We leverage the schemas defined in PowerSystems to create functions for defining Power Systems Analysis Problems. 

## Package Setup

This notebook requires Julia 1.1 and uses the environment setup in the subfolder `env`. You can setup the environment with the next two cells.

The environment should look like:
```julia
    Status `~/Documents/repos/Examples/env/Project.toml`
  [5ae59095] Colors v0.9.5
  [41994980] D3TypeTrees v0.1.1
  [a93c6f00] DataFrames v0.18.0
  [e2685f51] ECOS v0.9.4
  [60bf3e95] GLPK v0.9.1
  [2030c09a] InfrastructureModels v0.2.0 #moi-2 (https://github.com/lanl-ansi/InfrastructureModels.jl.git)
  [b6b21f68] Ipopt v0.5.4
  [4076af6c] JuMP v0.19.0+ #master (https://github.com/JuliaOpt/JuMP.jl.git)
  [51fcb6bd] NamedColors v0.2.0
  [774612a8] ParameterJuMP v0.0.1 #dfb1e3c (https://github.com/JuliaStochOpt/ParameterJuMP.jl.git)
  [f0f68f2c] PlotlyJS v0.12.3
  [91a5bcdd] Plots v0.24.0
  [c36e90e8] PowerModels v0.10.0 #moi-2 (https://github.com/lanl-ansi/PowerModels.jl.git)
  [e690365d] PowerSimulations v0.1.0 #master (https://github.com/nrel/powersimulations.jl)
  [bcd98974] PowerSystems v0.3.0
  [9e3dc215] TimeSeries v0.14.1
```
Execute the next code block to activate the necessary environment.

In [1]:
] activate env; instantiate

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`


In [2]:
] status

    Status `~/Documents/repos/Examples/env/Project.toml`
  [5ae59095] Colors v0.9.5
  [41994980] D3TypeTrees v0.1.1
  [a93c6f00] DataFrames v0.18.0
  [e2685f51] ECOS v0.9.4
  [60bf3e95] GLPK v0.9.1
  [2030c09a] InfrastructureModels v0.2.0 #moi-2 (https://github.com/lanl-ansi/InfrastructureModels.jl.git)
  [b6b21f68] Ipopt v0.5.4
  [4076af6c] JuMP v0.19.0+ #master (https://github.com/JuliaOpt/JuMP.jl.git)
  [51fcb6bd] NamedColors v0.2.0
  [774612a8] ParameterJuMP v0.0.1 #dfb1e3c (https://github.com/JuliaStochOpt/ParameterJuMP.jl.git)
  [f0f68f2c] PlotlyJS v0.12.3
  [91a5bcdd] Plots v0.24.0
  [c36e90e8] PowerModels v0.10.0 #moi-2 (https://github.com/lanl-ansi/PowerModels.jl.git)
  [e690365d] PowerSimulations v0.1.0 #master (https://github.com/nrel/powersimulations.jl)
  [bcd98974] PowerSystems v0.3.0
  [9e3dc215] TimeSeries v0.14.1


In [3]:
# Modeling Packages
using PowerSystems; # Power System Data Model
using PowerSimulations; # Power System Modeling
using JuMP;         # Optimization Modeling

### can use Xpress if you have a local license
using Xpress;          # Optimization Solver
xpress_optimizer = with_optimizer(Xpress.Optimizer, OUTPUTLOG=1,MIPTOL=0.00001)

### otherwise, use GLPK
#using GLPK;
#glpk_optimizer = with_optimizer(GLPK.Optimizer)


┌ Info: Recompiling stale cache file /Users/cbarrows/.julia/compiled/v1.1/Xpress/1jRVj.ji for Xpress [9e70acf3-d6c9-5be6-b5bd-4e2c73e3e054]
└ @ Base loading.jl:1184
┌ Info: Xpress: Found license file /Users/cbarrows/Applications/xpressmp/bin/xpauth.xpr
└ @ Xpress /Users/cbarrows/.julia/packages/Xpress/Jdi4R/src/xprs_userlic.jl:42
┌ Info: Xpress: Development license detected.
└ @ Xpress /Users/cbarrows/.julia/packages/Xpress/Jdi4R/src/xprs_userlic.jl:87


In [4]:
const PSI = PowerSimulations;
const PSY = PowerSystems;

In [5]:
# Result Inspection Packages
using DataFrames;

# RTS Preloads

In [6]:

BASE_DIR = abspath(joinpath(dirname(Base.find_package("PowerSystems")), ".."))
DATA_DIR = joinpath(BASE_DIR, "data")
RTS_GMLC_DIR = joinpath(DATA_DIR, "RTS_GMLC")



"/Users/cbarrows/.julia/packages/PowerSystems/Biewz/data/RTS_GMLC"

In [8]:

cdm_dict = PSY.csv2ps_dict(RTS_GMLC_DIR, 100.0)
delete!(cdm_dict,"dcline")
cdm_dict["gen"]["Renewable"]["PV"] = Dict{String,Any}()
cdm_dict["gen"]["Renewable"]["RTPV"] = Dict{String,Any}()
cdm_dict["gen"]["Renewable"]["WIND"] = Dict{String,Any}()
cdm_dict["gen"]["Hydro"] = Dict{String,Any}()
delete!(cdm_dict["forecast"]["DA"],"gen")
cdm_dict["forecast"]["DA"]["load"] = cdm_dict["forecast"]["DA"]["load"][1:2,:]
PSY.assign_ts_data(cdm_dict,cdm_dict["forecast"]["DA"])

sys_rts = PSY.System(cdm_dict)


┌ Info: Parsing csv files in Reserves ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/parsers/cdm_parser.jl:34
┌ Info: Parsing csv data in DAY_AHEAD_regional_Flex_Down.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/parsers/cdm_parser.jl:38
┌ Info: Parsing csv data in DAY_AHEAD_regional_Flex_Up.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/parsers/cdm_parser.jl:38
┌ Info: Parsing csv data in DAY_AHEAD_regional_Reg_Down.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/parsers/cdm_parser.jl:38
┌ Info: Parsing csv data in DAY_AHEAD_regional_Reg_Up.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/parsers/cdm_parser.jl:38
┌ Info: Parsing csv data in DAY_AHEAD_regional_Spin_Up_R1.csv ...
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/parsers/cdm_parser.jl:38
┌ Info: Parsing csv data in DAY_AHEAD_regional_Spin_Up_R

┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/Biewz/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/PV/DAY_AHEAD_pv.csv for 113_PV_1
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/Biewz/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/PV/DAY_AHEAD_pv.csv for 319_PV_1
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/Biewz/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/PV/DAY_AHEAD_pv.csv for 215_PV_1
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/Biewz/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/PV/DAY_AHEAD_pv.csv for 10

┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/Biewz/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/RTPV/DAY_AHEAD_rtpv.csv for 118_RTPV_7
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/Biewz/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/RTPV/DAY_AHEAD_rtpv.csv for 118_RTPV_8
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/Biewz/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/RTPV/DAY_AHEAD_rtpv.csv for 118_RTPV_9
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/Biewz/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/RTPV/DAY

┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/Biewz/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/RTPV/DAY_AHEAD_rtpv.csv for 313_RTPV_9
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/Biewz/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/RTPV/DAY_AHEAD_rtpv.csv for 313_RTPV_10
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/Biewz/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/RTPV/DAY_AHEAD_rtpv.csv for 313_RTPV_11
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/Biewz/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/RTPV/D

┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/Biewz/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Hydro/REAL_TIME_hydro.csv for 122_HYDRO_5
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/Biewz/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Hydro/REAL_TIME_hydro.csv for 122_HYDRO_6
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/Biewz/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Hydro/REAL_TIME_hydro.csv for 201_HYDRO_4
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/Biewz/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Hydro/REAL_TIME_hydro.csv for 215_HYDRO

┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/Biewz/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/RTPV/REAL_TIME_rtpv.csv for 118_RTPV_3
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/Biewz/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/RTPV/REAL_TIME_rtpv.csv for 118_RTPV_4
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/Biewz/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/RTPV/REAL_TIME_rtpv.csv for 118_RTPV_5
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/Biewz/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/RTPV/REA

┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/Biewz/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/RTPV/REAL_TIME_rtpv.csv for 313_RTPV_12
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/Biewz/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/RTPV/REAL_TIME_rtpv.csv for 320_RTPV_1
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/Biewz/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/RTPV/REAL_TIME_rtpv.csv for 320_RTPV_2
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/parsers/cdm_parser.jl:81
┌ Info: parsing timeseries data in /Users/cbarrows/.julia/packages/PowerSystems/Biewz/data/RTS_GMLC/../forecasts/RTS_GMLC_forecasts/gen/Renewable/RTPV/RE

┌ Warning: key 'dcline' not found in PowerSystems dictionary, this will result in an empty DCLines array
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/parsers/dict_to_struct.jl:52
┌ Error: Model doesn't contain a slack bus
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/utils/IO/system_checks.jl:51
┌ Warning: The angle limits provided is larger than 3π/2 radians.
│  PowerSystems inferred the data provided in degrees and will transform it to radians
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/utils/IO/branchdata_checks.jl:19
┌ Warning: The angle limits provided is larger than 3π/2 radians.
│  PowerSystems inferred the data provided in degrees and will transform it to radians
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/utils/IO/branchdata_checks.jl:19
┌ Warning: The angle limits provided is larger than 3π/2 radians.
│  PowerSystems inferred the data provided in degrees and will trans

┌ Warning: The angle limits provided is larger than 3π/2 radians.
│  PowerSystems inferred the data provided in degrees and will transform it to radians
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/utils/IO/branchdata_checks.jl:19
┌ Warning: The angle limits provided is larger than 3π/2 radians.
│  PowerSystems inferred the data provided in degrees and will transform it to radians
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/utils/IO/branchdata_checks.jl:19
┌ Warning: The angle limits provided is larger than 3π/2 radians.
│  PowerSystems inferred the data provided in degrees and will transform it to radians
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/utils/IO/branchdata_checks.jl:19
┌ Warning: The angle limits provided is larger than 3π/2 radians.
│  PowerSystems inferred the data provided in degrees and will transform it to radians
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/u

System:
   buses: Bus[Bus(name="Abel"), Bus(name="Adams"), Bus(name="Adler"), Bus(name="Agricola"), Bus(name="Aiken"), Bus(name="Alber"), Bus(name="Alder"), Bus(name="Alger"), Bus(name="Ali"), Bus(name="Allen")  …  Bus(name="Chifa"), Bus(name="Chuhsi"), Bus(name="Clark"), Bus(name="Clay"), Bus(name="Clive"), Bus(name="Cobb"), Bus(name="Cole"), Bus(name="Comte"), Bus(name="Curie"), Bus(name="Curtiss")]
   generators: 
     GenClasses(T:73,R:0,H:0):
   thermal: ThermalDispatch[ThermalDispatch(name="322_CT_6"), ThermalDispatch(name="321_CC_1"), ThermalDispatch(name="202_STEAM_3"), ThermalDispatch(name="315_STEAM_1"), ThermalDispatch(name="223_CT_4"), ThermalDispatch(name="123_STEAM_2"), ThermalDispatch(name="213_CT_1"), ThermalDispatch(name="223_CT_6"), ThermalDispatch(name="313_CC_1"), ThermalDispatch(name="202_CT_2")  …  ThermalDispatch(name="216_STEAM_1"), ThermalDispatch(name="315_CT_6"), ThermalDispatch(name="201_CT_2"), ThermalDispatch(name="201_STEAM_3"), ThermalDispatch(name="201_

# RTS-GMLC

## Define the model

In [9]:
ED_copperplate = PSI.EconomicDispatch(sys_rts, PSI.CopperPlatePowerModel; optimizer = xpress_optimizer, parameters=false)
ED_copperplate.devices[:ThermalGenerators] = PowerSimulations.DeviceModel(ThermalGen, PSI.ThermalDispatchNoMin)
PSI.build_op_model!(ED_copperplate; optimizer = xpress_optimizer, parameters=false)

In [10]:
sys_rts.generators.thermal[1].tech

TechThermal:
   activepower: 0.55
   activepowerlimits: (min = 0.22, max = 0.55)
   reactivepower: -0.0973
   reactivepowerlimits: (min = -0.15, max = 0.19)
   ramplimits: (up = 2.22, down = 2.22)
   timelimits: (up = 2.2, down = 2.2)

## Run the Simulation

In [12]:
solve_op_model!(ED_copperplate)

┌ Warning: MathOptInterface.VariablePrimalStart() is not supported by MathOptInterface.Bridges.LazyBridgeOptimizer{Xpress.Optimizer,MathOptInterface.Utilities.UniversalFallback{MathOptInterface.Bridges.AllBridgedConstraints{Float64}}}. This 
│   information will be discarded. = information will be discarded.
└ @ MathOptInterface.Utilities /Users/cbarrows/.julia/packages/MathOptInterface/C3lip/src/Utilities/copy.jl:133


Minimizing LP 
Original problem has:
      1170 rows          584 cols         1752 elements
Presolved problem has:
         2 rows          234 cols          234 elements

   Its         Obj Value      S   Ninf  Nneg   Sum Dual Inf  Time
     0      -2420.468218      D      2     0        .000000     0
     2       368007.1730      D      0     0        .000000     0
Uncrunching matrix
Optimal solution found
Dual solved problem
  2 simplex iterations in 0s

Final objective                         : 3.680071730365644e+05
  Max primal violation      (abs / rel) :       0.0 /       0.0
  Max dual violation        (abs / rel) :       0.0 /       0.0
  Max complementarity viol. (abs / rel) :       0.0 /       0.0
All values within tolerances


PowerSimulations.OpertationModelResults(Dict(:Pth=>2×73 DataFrame. Omitted printing of 68 columns
│ Row │ 322_CT_6 │ 321_CC_1 │ 202_STEAM_3 │ 315_STEAM_1 │ 223_CT_4 │
│     │ Float64  │ Float64  │ Float64     │ Float64     │ Float64  │
├─────┼──────────┼──────────┼─────────────┼─────────────┼──────────┤
│ 1   │ 0.33     │ 1.23     │ 0.46        │ 0.0         │ 0.11     │
│ 2   │ 0.33     │ 1.23     │ 0.46        │ 0.0         │ 0.11     │), Dict(:ED=>3.68007e5), Dict{Symbol,Any}(:dual_status=>FEASIBLE_POINT,:primal_status=>FEASIBLE_POINT,:termination_status=>OPTIMAL))